https://www.geeksforgeeks.org/python-sqlite/


In [25]:
import sqlite3;

conn = sqlite3.connect(r"C:\Users\capnf\Downloads\plum.sqlite")

c = conn.cursor()

In [29]:
sql_query = """SELECT name FROM sqlite_master 
    WHERE type='table';"""
 

cursor = conn.cursor()
     

cursor.execute(sql_query)
print("List of tables\n")
     

print(cursor.fetchall())

List of tables

[('Stroke',), ('StrokeMetadata',), ('User',), ('SyncState',), ('UpgradedNote',), ('Media',), ('Note',), ('Insight',), ('Tag',)]


In [67]:
cursor = conn.execute("SELECT * from Note")
 

names = list(map(lambda x: x[0], cursor.description))

print(names)

a = []

for row in cursor:
    a.append(row)
    print(row)


row = a[0][0]


sorok = row.split("\n")
print(sorok[1])
sor = sorok[1].replace("\\id="," ")
sor = sorok[3][40:]
print(sor)


['Text', 'WindowPosition', 'IsOpen', 'IsAlwaysOnTop', 'CreationNoteIdAnchor', 'Theme', 'IsFutureNote', 'RemoteId', 'ChangeKey', 'LastServerVersion', 'RemoteSchemaVersion', 'IsRemoteDataInvalid', 'PendingInsightsScan', 'Type', 'Id', 'ParentId', 'CreatedAt', 'DeletedAt', 'UpdatedAt']
('\\id=6c8cad39-2189-4aa1-9c3d-c8d9ca01c0fe \n\\id=7dfeee4d-050c-4a0f-8420-9f94402e9d8c\\l office\n\\id=99905254-32a8-4583-b94e-ec23dc0db87d acrobat dc pro\n\\id=d2688e83-6cb0-444a-84ea-0020903158da photoshop pro\n\\id=ce2f6a20-a0d9-4e7a-9feb-9a7c32ad0ee6\\l0 \n\\id=163a0b7a-6565-429c-b40b-0704dfabefe3\\l vs studio', 'ManagedPosition=', 0, 0, None, 'Yellow', 0, 'AAkALgAAAAAAHYQDEapmEc2byACqAC-EWg0Aj8aDvXr_oUaaWyDzQ-pe9QAB6BbyFAAA', 'CQAAABYAAACPxoO9ev6hRppbIPND+l71AAHoFH/c', '{\r\n  "id": "AAkALgAAAAAAHYQDEapmEc2byACqAC-EWg0Aj8aDvXr_oUaaWyDzQ-pe9QAB6BbyFAAA",\r\n  "changeKey": "CQAAABYAAACPxoO9ev6hRppbIPND+l71AAHoFH/c",\r\n  "createdWithLocalId": "d04976a6-ec0d-47db-8599-76aac9688633",\r\n  "color": 1,\r\n  

In [23]:
import os
import sqlite3
import pandas as pd
# Path to the SQLite database, it is copied to Downloads folder before the code is executed. replace the <your_username> to your username.
db_path = r'C:\Users\capnf\Downloads\plum.sqlite'  # 
# Path for the CSV file
csv_path = os.path.join(os.getcwd(), 'sticky_notes_backup.csv')
# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
# Retrieve all table names in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
# Print all table names
print("Tables in the database:")
for table in tables:
    print(table[0])
# Inspect each table schema
for table in tables:
    print(f"\nSchema for table {table[0]}:")
    cursor.execute(f"PRAGMA table_info({table[0]})")
    columns = cursor.fetchall()
    for column in columns:
        print(column)
# Function to identify the most likely table for sticky notes content
def find_notes_table(tables):
    possible_names = ['Notes', 'Note', 'StickyNotes', 'StickyNote']
    for table in tables:
        if table[0] in possible_names:
            return table[0]
    return None
# Identify the table name
table_name = find_notes_table(tables)
if table_name:
    # Query the data from the correct table
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    # Export the DataFrame to a CSV file
    df.to_csv(csv_path, index=False)
    print(f"Export completed successfully! File saved as {csv_path}")
else:
    print("No suitable table found for sticky notes content.")
# Close the database connection
conn.close()

Tables in the database:
Stroke
StrokeMetadata
User
SyncState
UpgradedNote
Media
Note
Insight
Tag

Schema for table Stroke:
(0, 'Data', 'BLOB', 0, None, 0)
(1, 'FormatVersion', 'INTEGER', 0, None, 0)
(2, 'Type', 'varchar', 0, None, 0)
(3, 'Id', 'varchar', 1, None, 1)
(4, 'ParentId', 'varchar', 0, None, 0)
(5, 'CreatedAt', 'bigint', 0, None, 0)
(6, 'DeletedAt', 'bigint', 0, None, 0)
(7, 'UpdatedAt', 'bigint', 0, None, 0)

Schema for table StrokeMetadata:
(0, 'InsightId', 'varchar', 0, None, 0)
(1, 'OffsetAngle', 'float', 0, None, 0)
(2, 'RotationPointX', 'float', 0, None, 0)
(3, 'RotationPointY', 'float', 0, None, 0)
(4, 'OffsetX', 'float', 0, None, 0)
(5, 'OffsetY', 'float', 0, None, 0)
(6, 'ReminderId', 'varchar', 0, None, 0)
(7, 'StrokeId', 'varchar', 0, None, 0)
(8, 'Type', 'varchar', 0, None, 0)
(9, 'Id', 'varchar', 1, None, 1)
(10, 'ParentId', 'varchar', 0, None, 0)
(11, 'CreatedAt', 'bigint', 0, None, 0)
(12, 'DeletedAt', 'bigint', 0, None, 0)
(13, 'UpdatedAt', 'bigint', 0, None, 

In [24]:
import csv
import re
from datetime import datetime
from docx import Document
# Function to extract text data and last modified date
def extract_text_data(csv_file):
    text_data = []
    # Read the CSV file
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for row in reader:
            row_text = ''.join(row)  # Combine all columns into one string
            text_matches = re.findall(r'"text":\s*"([^"]*)"', row_text, re.DOTALL)
            date_matches = re.findall(r'"lastModified":\s*"([^"]*)"', row_text)
            if text_matches and date_matches:
                for text in text_matches:
                    last_modified = date_matches[0]  # Assume the first date match is the correct one
                    text_data.append((text, last_modified))
    # Sort text data by lastModified date in descending order
    text_data.sort(key=lambda x: datetime.strptime(x[1], '%Y-%m-%dT%H:%M:%SZ'), reverse=True)
    return text_data
# Function to write the extracted text data to a Word document
def write_to_word(text_data, output_word_file):
    doc = Document()
    last_date = None
    
    for text, last_modified in text_data:
        if last_date != last_modified:
            if last_date is not None:
                doc.add_paragraph('-' * 40)  # Add a line of dashes
            last_date = last_modified
        doc.add_paragraph(text)
    
    doc.save(output_word_file)
# Example usage. replace the <your_username> to your username.
csv_file_path = r'C:\Users\capnf\source\pyrepos\StickyNotesConverter\sticky_notes_backup.csv'  # Replace with the actual path
extracted_texts = extract_text_data(csv_file_path)
# Write the extracted texts to a new Word document. replace the <your_username> to your username.
output_word_file_path = r'C:\Users\capnf\Downloads\extracted_texts_third.docx'  # Replace with the actual path
write_to_word(extracted_texts, output_word_file_path)
print(f"Extracted texts have been written to {output_word_file_path}")

Extracted texts have been written to C:\Users\capnf\Downloads\extracted_texts_third.docx
